<a href="https://colab.research.google.com/github/chodizzle/portfolio1/blob/main/projects/priority_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149803 sha256=6d61fdc1e60944a057ecefd4d580ce4ea20ac72852340ba7f0715d837c28a619
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import pandas as pd
import numpy as np
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.util import ngrams
from collections import Counter

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/UntitledChart_2021-5-17_1458.csv', index_col='item_id')

In [ ]:
# cats = df['category'].unique()

In [ ]:
# cats_sample = cats[0:2]

df_concat = pd.DataFrame()
counter = len(cats)

for cat in cats:
  print('Processing:', cat)
  counter = counter - 1
  print(counter, 'left')
  df_temp = df[df['category'] == cat]

  list = []
  for name in df_temp['name'].values:
    name = name.strip()
    name = name.lower()
    list.append(name)
    
  vectorizer = CountVectorizer().fit_transform(list)
  vectors = vectorizer.toarray()
  
  csim = cosine_similarity(vectors)
  
  csim_mean = []
  for c in csim:
    csim_mean.append(c.mean())
  df_temp['csim'] = csim_mean
  
  df_concat = df_concat.append(df_temp)

In [ ]:
# df_concat['item_fuzz_score'] = df_concat[['name', 'category']].apply(lambda x: fuzz.token_set_ratio(x[0], x[1]), axis=1)

In [ ]:
# df_concat['csim_mean'] = df_concat.groupby('category')['csim'].transform('mean')
# df_concat['fuzz_mean'] = df_concat.groupby('category')['item_fuzz_score'].transform('mean')

In [ ]:
df_concat.to_csv('scored_meats.csv',  index=True, index_label='item_id')

In [ ]:
!pwd

/content/drive/My Drive


In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
df = pd.read_csv('scored_meats.csv', index_col='item_id')
df_cats = pd.read_csv('cat_metrics.csv')
df_keywords = pd.read_csv('keywords.csv')

In [ ]:
df.head(), '\n' ,df_cats.head(), '\n', df_keywords.head()

(                                       name   category  ... csim_mean  fuzz_mean
 item_id                                                 ...                     
 7739819  adobo,con pimienta,red top 12/28oz  Al Pastor  ...  0.078827   0.460678
 2015836                       al barino2017  Al Pastor  ...  0.078827   0.460678
 5435305            pork shlor slco al pstor  Al Pastor  ...  0.078827   0.460678
 2239490                       al barino2017  Al Pastor  ...  0.078827   0.460678
 6965346                              paster  Al Pastor  ...  0.078827   0.460678
 
 [5 rows x 7 columns],
 '\n',
           category  csim_mean  fuzz_mean  cat_count
 0   Chicken Breast   0.173539   0.664589      15499
 1            Bacon   0.176905   0.854201      14381
 2      Ground Beef   0.134850   0.554369      10030
 3  Italian Sausage   0.159786   0.602094      10022
 4    Chicken Thigh   0.168362   0.698305       8899,
 '\n',
     keywords  counts        category
 0  ('brst',)     488  Chicken

In [ ]:
df_cats.describe()

,csim_mean,fuzz_mean,cat_count
count,364.000000,364.000000,364.000000
mean,0.340174,0.655928,845.211538
std,0.276337,0.149533,1926.179592
min,0.036804,0.166601,1.000000
25%,0.158812,0.556968,11.000000
50%,0.214711,0.641294,116.500000
75%,0.424509,0.736356,712.000000
max,1.000000,1.000000,15499.000000


In [ ]:
# df_cats = df[['category', 'csim_mean', 'fuzz_mean', 'cat_count']].drop_duplicates().reset_index().drop('item_id', axis=1)
# no longer needed after save

In [ ]:
df_cats.to_csv('cat_metrics.csv',  index=False)
# last saved after sorting

In [ ]:
# df_cats.sort_values(['cat_count', 'csim_mean', 'fuzz_mean'], ascending=False, inplace=True)

In [ ]:
# df_cats.reset_index(drop=True, inplace=True)

In [ ]:
# first try. Mixed results. Top 20 was based solely off of cat size.
# IGNORE ME
top_20 = df_cats.loc[:20, 'category']

df_keywords = pd.DataFrame()

for cat in top_20:
  mask = (df['category'] == cat) & (df['csim'] < df['csim_mean']) & (df['item_fuzz_score'] < .8 * df['fuzz_mean'])

  words = ''
  for names in df[mask]['name'].values:
    words += names

  tokens = words.split()
  unigrams = Counter(ngrams(tokens, 1)).most_common(25)
  bigrams = Counter(ngrams(tokens, 2)).most_common(25)
  trigrams = Counter(ngrams(tokens, 3)).most_common(25)

  grams = unigrams + bigrams + trigrams

  df_temp = pd.DataFrame(grams, columns=['keywords', 'counts'])
  df_temp['category'] = cat
  
  df_keywords = df_keywords.append(df_temp)

In [ ]:
# getting categories with relatively large size and low fuzz match
count_filt = df_cats['cat_count'] > df_cats['cat_count'].quantile(.75)
fuzz_filt = df_cats['fuzz_mean'] < df_cats['fuzz_mean'].quantile(.25)
cats2 = df_cats[count_filt & fuzz_filt]['category']
cats2.to_csv('test_cats.csv')

In [ ]:
df_keywords = pd.DataFrame()

for cat in cats2:
  mask = (df['category'] == cat) & (df['item_fuzz_score'] < df['fuzz_mean']) & (df['csim'] > df['csim_mean'])

  words = ''
  for names in df[mask]['name'].values:
    words += names

  tokens = words.split()
  try:
    unigrams = list(set(ngrams(tokens, 1)))
  except:
    unigrams = []
  try:
    bigrams = list(set(ngrams(tokens, 2)))
  except:
    bigrams = []
  try:
    trigrams = list(set(ngrams(tokens, 3)))
  except:
    trigrams = []

  if len(unigrams) > 25: unigrams = unigrams[0:26]
  if len(bigrams) > 25: bigrams = bigrams[0:26]
  if len(trigrams) > 25: trigrams = trigrams[0:26]
  grams = unigrams + bigrams + trigrams

  df_temp = pd.DataFrame(grams, columns=['a', 'b', 'c'])
  df_temp['category'] = cat
  df_temp.fillna('', inplace=True)
  df_temp['keywords'] = df_temp[['a', 'b', 'c']].agg(', '.join, axis=1)
  df_temp.drop(['a', 'b', 'c'], axis=1, inplace=True)
  df_keywords = df_keywords.append(df_temp)

df_keywords.reset_index(drop=True, inplace=True)

In [ ]:
df_keywords

,category,keywords
0,Ground Beef,"refbeef,phily, ,"
1,Ground Beef,"81/19frank, ,"
2,Ground Beef,"steer, ,"
3,Ground Beef,"81/19asian, ,"
4,Ground Beef,"0x1, ,"
...,...,...
2242,Pork Chop Center Cut,"loin, baby, back"
2243,Pork Chop Center Cut,"bone, in, fznpork"
2244,Pork Chop Center Cut,"boneless, c/c, pork"
2245,Pork Chop Center Cut,"pork, &, beanspork"


In [ ]:
# df_keywords.to_csv('keywords2.csv', index=False)

CHICKEN BREAST TEST


In [ ]:
df_chixbrst = df[df['category'] == 'Chicken Breast']

In [ ]:
df_chixbrst['fuzz1'] = df_chixbrst.loc[:, ['name', 'category']].apply(lambda x: fuzz.token_set_ratio(x[0], x[1]), axis=1)

In [ ]:
mask = (df_chixbrst['fuzz1'] > 25) & (df_chixbrst['fuzz1'] < 75)

In [ ]:
df_chixbrst[mask]

In [ ]:
df_chixbrst['cat_mean'] = df_chixbrst['fuzz1'].mean()

In [ ]:
df_chixbrst.sort_values('fuzz1', ascending=False)

In [ ]:
names = df_chixbrst['name'].values
# line = ''
# for name in names:
#   print(name)
#   name = name.strip()
#   name = name.lower()
#   line += name
# tokens = line.split()
# tokens

# not what I want...

In [ ]:
list = []
for name in df_chixbrst['name'].values:
  name = name.strip()
  name = name.lower()
  list.append(name)
list

vectorizer = CountVectorizer().fit_transform(list)
vectors = vectorizer.toarray()
# vectors

csim = cosine_similarity(vectors)
csim

In [ ]:
csim_mean = []
for c in csim:
  csim_mean.append(c.mean())

csim_mean

In [ ]:
df_chixbrst['csim'] = csim_mean

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.scatter(x=df_chixbrst['fuzz1'], y=df_chixbrst['csim'])

END CHICKEN BREAST TEST

RESUME REGULAR PROGRAMMING


In [ ]:
df1 = df.groupby('category').filter(lambda x: len(x) > 20)